- Check the movements with best vs worst predictions
- Check the types of movements that are the best / worst predicted.

In [1]:
import json
import pandas as pd

In [25]:
# Opening the JSON file
with open('results_2_.json', 'r') as f:
    data = json.load(f)

# Extract the movements
movements = data["Predictions"]

# Names of the columns
columns = ["T_x", "T_y", "T_z", "T_roll", "T_pitch", "T_yaw", "T_lin", "T_ang", 
           "P_x", "P_y", "P_z", "P_roll", "P_pitch", "P_yaw", "P_lin", "P_ang", 
           "D_x", "D_y", "D_z", "D_roll", "D_pitch", "D_yaw", "D_lin", "D_ang"]

# Creating the DataFrame
rows = []
for movement in movements:
    row = movement["Ground Truth"] + movement["Prediction"] + movement["Difference"]
    rows.append(row)

df = pd.DataFrame(rows, columns=columns)
df = df.apply(pd.to_numeric)

## 1. Components of movements that are the best / worst predicted

In [27]:
# Components of the movements
components = ["x", "y", "z", "roll", "pitch", "yaw", "lin", "ang"]

# Creating a DataFrame to show some results
df_results = pd.DataFrame({
    "Component": components,
    "Sum of Squared Differences": [(df[f"D_{c}"].abs()).sum()  for c in components],
    "Max absolute error": [df[f"D_{c}"].abs().max() for c in components],
    "Relative error": [round((df[f"D_{c}"] / df[f"T_{c}"]).abs()[df[f"T_{c}"] != 0].mean()*100,2) for c in components],
})

# Printing the DataFrame
print(df_results)


  Component  Sum of Squared Differences  Max absolute error  Relative error
0         x                       23.52                0.37           34.04
1         y                       17.48                0.33           21.38
2         z                        6.24                0.09             NaN
3      roll                        5.17                0.12             NaN
4     pitch                        3.03                0.09             NaN
5       yaw                       46.91                2.60           42.69
6       lin                       23.99                0.42            9.68
7       ang                       65.13                3.99          712.37


We don't take into account z, roll and pitch because the ground truth is always 0. But the fact that we still have some errors, shows that our model didn't remember thoses parameters but tried to estimate them instead. It's an indicator that goes against an overfitted model.

We can see that our model struggles most with forecasting the angular velocity. 

## 2. Best predicted movements

In [29]:
df["Total_Error"] = df[[f"D_{c}" for c in components]].abs().sum(axis=1)
best_predicted = df.nsmallest(int(len(df) * 0.05), "Total_Error")
print(best_predicted[[col for col in df.columns if col.startswith("T_")]])

      T_x   T_y  T_z  T_roll  T_pitch  T_yaw  T_lin  T_ang
207 -0.65  0.14  0.0     0.0      0.0   0.95   1.08   0.05
58   0.05 -0.26  0.0     0.0      0.0  -1.57   0.56  -0.02
59   0.02 -0.30  0.0     0.0      0.0  -1.57   0.56  -0.02
187 -1.20 -0.36  0.0     0.0      0.0   0.18   1.08   0.02
186 -1.16 -0.37  0.0     0.0      0.0   0.18   1.08  -0.02
185 -1.12 -0.37  0.0     0.0      0.0   0.18   1.08  -0.02
75   0.37 -0.37  0.0     0.0      0.0  -0.37   0.56  -0.02
76   0.40 -0.35  0.0     0.0      0.0  -0.37   0.56  -0.02
208 -0.69  0.16  0.0     0.0      0.0   0.96   1.08   0.01
77   0.44 -0.33  0.0     0.0      0.0  -0.37   0.56  -0.02
78   0.47 -0.31  0.0     0.0      0.0  -0.37   0.56  -0.02
79   0.51 -0.28  0.0     0.0      0.0  -0.37   0.56  -0.02


In [30]:
df["Total_Error"] = df[[f"D_{c}" for c in components]].abs().sum(axis=1)
best_predicted = df.nlargest(int(len(df) * 0.05), "Total_Error")
print(best_predicted[[col for col in df.columns if col.startswith("T_")]])

      T_x   T_y  T_z  T_roll  T_pitch  T_yaw  T_lin  T_ang
27  -0.04 -1.95  0.0     0.0      0.0   0.42   1.08  -3.54
61  -0.04 -0.37  0.0     0.0      0.0  -1.54   0.56   4.14
165 -0.30 -0.33  0.0     0.0      0.0  -0.04   1.08   1.41
31  -0.18 -2.06  0.0     0.0      0.0   2.41   1.08  -0.19
192 -1.41 -0.36  0.0     0.0      0.0  -2.40   1.08  -0.02
4   -0.02 -0.36  0.0     0.0      0.0   1.03   1.08  -3.39
168 -0.42 -0.30  0.0     0.0      0.0  -2.38   1.08   0.15
179 -0.87 -0.38  0.0     0.0      0.0   0.18   1.08  -0.06
193 -1.45 -0.36  0.0     0.0      0.0  -2.40   1.08  -0.02
28  -0.06 -2.02  0.0     0.0      0.0   0.09   1.08  -1.66
5    0.01 -0.40  0.0     0.0      0.0   0.72   1.08  -1.59
199 -1.70 -0.36  0.0     0.0      0.0  -2.40   1.08  -0.02


Indeed, our model struggles more when forecasting movements with a higher angular velocity.